Merge the log2FC of the different DEA into 1 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from upsetplot import plot
import numpy as np

In [2]:
BC_dea = pd.read_csv('BC/DEA_Results_ihw.tsv', sep='\t')
CF_dea = pd.read_csv('CF/DEA_Results_ihw.tsv', sep='\t')
MI_168_dea = pd.read_csv('MI_168hpi/DEA_Results_ihw.tsv', sep='\t')
MI_336_dea = pd.read_csv('MI_336hpi/DEA_Results_ihw.tsv', sep='\t')
PI_dea = pd.read_csv('PI/DEA_Results_ihw.tsv', sep='\t')
PM_dea = pd.read_csv('PM/DEA_Results_ihw.tsv', sep='\t')
PS23_dea = pd.read_csv('PS23/DEA_Results_ihw.tsv', sep='\t')

In [3]:
conditions = ['Pinfestans','Cfulvum','PSTVd_S23','PSTVd_M','Bcinerea','Mincognita_7dpi','Mincognita_14dpi']
dfs = [PI_dea,CF_dea,PS23_dea,PM_dea,BC_dea,MI_168_dea,MI_336_dea]

In [4]:
for i, df in enumerate(dfs):
    sample_name = conditions[i]
    # Rename the columns and leave the index (gene names) unchanged
    df.columns = [f"{col}_{sample_name}" for col in df.columns]

In [5]:
merged = pd.concat(dfs, axis=1)

In [6]:
merged

,baseMean_Pinfestans,log2FoldChange_Pinfestans,lfcSE_Pinfestans,stat_Pinfestans,pvalue_Pinfestans,padj_Pinfestans,weight_Pinfestans,baseMean_Cfulvum,log2FoldChange_Cfulvum,lfcSE_Cfulvum,...,pvalue_Mincognita_7dpi,padj_Mincognita_7dpi,weight_Mincognita_7dpi,baseMean_Mincognita_14dpi,log2FoldChange_Mincognita_14dpi,lfcSE_Mincognita_14dpi,stat_Mincognita_14dpi,pvalue_Mincognita_14dpi,padj_Mincognita_14dpi,weight_Mincognita_14dpi
mRNA:Solyc00g160260.1.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
mRNA:Solyc00g160270.1.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
mRNA:Solyc00g500003.1.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.681241,-0.240833,3.989260,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
mRNA:Solyc00g500004.1.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
mRNA:Solyc00g500005.1.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mRNA:Solyc12g100330.2.1,271.742892,1.515504,0.231056,6.559033,5.415799e-11,3.134909e-10,1.150208,812.904878,0.593120,0.336327,...,0.007892,0.015667,1.133710,160.546929,-0.210833,0.179258,-1.176143,0.239538,0.610731,0.481703
mRNA:Solyc12g100340.1.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.835258,-4.438186,3.133445,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
mRNA:Solyc12g100360.1.1,1979.994168,0.039386,0.086263,0.456579,6.479734e-01,1.000000e+00,0.724581,6021.790243,0.229788,0.139844,...,0.915582,0.876961,1.213279,351.358209,0.242920,0.060036,4.046267,0.000052,0.000175,0.965220
mRNA:Solyc12g100363.1.1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
merged_lFC=merged.filter(like='log2FoldChange')
merged_lFC=merged_lFC.fillna(0)

In [8]:
merged_lFC.to_csv('Merge_logfc_all.tsv', sep='\t')

In [9]:
merged_padj = merged.filter(like='padj')
merged_padj=merged_padj.fillna(1)

In [10]:
def get_groups_dea(row):
    groups = []
    cols = merged_padj.columns
    for col in cols:
        if row[col] <= 0.05:
            groups.append(col.removeprefix('padj_'))
    return ' & '.join(groups) if groups else None

In [11]:
merged_padj_groups = merged_padj.copy()

In [12]:
merged_padj_groups['Groups_DEA'] = merged_padj_groups.apply(get_groups_dea, axis=1)

In [13]:
merged_padj

,padj_Pinfestans,padj_Cfulvum,padj_PSTVd_S23,padj_PSTVd_M,padj_Bcinerea,padj_Mincognita_7dpi,padj_Mincognita_14dpi
mRNA:Solyc00g160260.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000
mRNA:Solyc00g160270.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000
mRNA:Solyc00g500003.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000
mRNA:Solyc00g500004.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000
mRNA:Solyc00g500005.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000
...,...,...,...,...,...,...,...
mRNA:Solyc12g100330.2.1,3.134909e-10,0.138714,0.048774,0.457645,2.773822e-03,0.015667,0.610731
mRNA:Solyc12g100340.1.1,1.000000e+00,0.532382,1.000000,1.000000,1.000000e+00,1.000000,1.000000
mRNA:Solyc12g100360.1.1,1.000000e+00,0.221788,0.291141,0.022829,4.386007e-20,0.876961,0.000175
mRNA:Solyc12g100363.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000


In [14]:
merged_padj_groups

,padj_Pinfestans,padj_Cfulvum,padj_PSTVd_S23,padj_PSTVd_M,padj_Bcinerea,padj_Mincognita_7dpi,padj_Mincognita_14dpi,Groups_DEA
mRNA:Solyc00g160260.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,None
mRNA:Solyc00g160270.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,None
mRNA:Solyc00g500003.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,None
mRNA:Solyc00g500004.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,None
mRNA:Solyc00g500005.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,None
...,...,...,...,...,...,...,...,...
mRNA:Solyc12g100330.2.1,3.134909e-10,0.138714,0.048774,0.457645,2.773822e-03,0.015667,0.610731,Pinfestans & PSTVd_S23 & Bcinerea & Mincognita...
mRNA:Solyc12g100340.1.1,1.000000e+00,0.532382,1.000000,1.000000,1.000000e+00,1.000000,1.000000,None
mRNA:Solyc12g100360.1.1,1.000000e+00,0.221788,0.291141,0.022829,4.386007e-20,0.876961,0.000175,PSTVd_M & Bcinerea & Mincognita_14dpi
mRNA:Solyc12g100363.1.1,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,None


In [15]:
merged_padj_groups.to_csv('./Merge_padj_groups_all.tsv', sep='\t')

In [10]:
#merged_padj.to_csv('Merge_padj_all.tsv', sep='\t')

In [16]:
merged_stat = merged.filter(like='stat')
merged_stat=merged_stat.fillna(0)

In [17]:
merged_stat.to_csv('Merge_stat_all.tsv', sep='\t')